<a href="https://colab.research.google.com/github/nhgowtham/hackathon_gg/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

Mounted at /content/drive
Note: using Google CoLab


In [1]:
!nvidia-smi

Wed Dec 17 06:43:52 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   68C    P8             12W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!git clone git@github.com:nhgowtham/hackathon_gg.git

Cloning into 'hackathon_gg'...
Host key verification failed.
fatal: Could not read from remote repository.

Please make sure you have the correct access rights
and the repository exists.


In [14]:
!ls drive

MyDrive


In [1]:
!pip install -U gdown

## Dataset download

In [ ]:
# https://drive.google.com/drive/folders/1u0z4pkZJN8Q00zt_QyEWHpckuFJ_z3iZ?usp=drive_link

#  Graphere dataset
!gdown --folder https://drive.google.com/drive/folders/1u0z4pkZJN8Q00zt_QyEWHpckuFJ_z3iZ -O data/